In [3]:
# %matplotlib inline

import sys, tables
import matplotlib
import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.dates as md
import matplotlib.gridspec as gridspec


In [4]:
f = tables.open_file(
    "/scr1/RadmonHd5/Fills2016/radmon5056.hd5")

In [8]:
# Checking Lumi
for leaf in f.walk_nodes(where='/',classname="Leaf"):
    if leaf.name == 'radmonraw':
        data =  f.get_node(where='/',name="radmonraw", classname="Leaf")
#         indxs = {}
#         for i, j in enumerate(data.colnames):
#             indxs[j] = i
#         print indxs
        print len(data)
        for row in range(0, len(data)):
            if row < 10:
                dtime = dt.datetime.fromtimestamp(data[row]['timestampsec']).strftime('%Y-%m-%d %H:%M:%S')
                print dtime
                print data[row]
            

7489
2016-06-30 01:44:42
(5056L, 275960L, 66L, 4L, 1467240282L, 802L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:44
(5056L, 275960L, 66L, 8L, 1467240284L, 317L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:45
(5056L, 275960L, 66L, 12L, 1467240285L, 767L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:47
(5056L, 275960L, 66L, 16L, 1467240287L, 167L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:48
(5056L, 275961L, 1L, 4L, 1467240288L, 782L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:50
(5056L, 275961L, 1L, 8L, 1467240290L, 304L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:51
(5056L, 275961L, 1L, 12L, 1467240291L, 718L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:53
(5056L, 275961L, 1L, 16L, 1467240293L, 242L, 76L, 4, 11, 101, 0, 11, 'LowLumi2015', 0.0, 0.0)
2016-06-30 01:44:54
(5056L, 275961L, 1L, 20L, 1467240294L, 668L, 76L, 4, 11, 101, 0

In [3]:
# plotting Lumi
for leaf in f.walk_nodes(where='/',classname="Leaf"):
    if leaf.name == 'radmonlumi':
        data =  f.get_node(where='/',name="radmonlumi", classname="Leaf")
        timestamps = np.empty(len(data))
        minusz = np.empty(len(data))
        minuszrms = np.empty(len(data))
        for index, item in enumerate(data):
            timestamps[index] = item['timestampsec']
            minusz[index] = item['plusz']
            minuszrms[index] = item['pluszrms']
            
        try:
            fig = plt.figure(figsize=(10, 5), dpi=96)
            fig.canvas.set_window_title('Lumi')
            dates = [dt.datetime.fromtimestamp(ts) for ts in timestamps]
            ax = plt.gca()
            ax.set_ylim([0.,50])
            plt.xticks(rotation = 45)
            plt.subplots_adjust(bottom = 0.3)
            xfmt = md.DateFormatter('%Y-%m-%d %H:%M')
            ax.xaxis.set_major_formatter(xfmt)
            plt.plot(dates, minusz, color='blue', fillstyle="full")
#             plt.errorbar(dates, minusz, yerr=minuszrms, color='blue', fillstyle="full")
            plt.show()
        
        except Exception as e:
            print e.__doc__
            print e.message
            raise      
        

#plotting rate

['fillnum', 'runnum', 'lsnum', 'nbnum', 'timestampsec', 'timestampmsec', 'totsize', 'publishnnb', 'datasourceid', 'algoid', 'channelid', 'payloadtype', 'rate', 'voltage', 'current', 'status', 'readouttime']

In [ ]:
id = 11 # unstable guy
for leaf in f.walk_nodes(where='/',classname="Leaf"):
    if leaf.name == 'radmonraw':
        data =  f.get_node(where='/',name="radmonraw", classname="Leaf")
        timestamps = []
        rate = []
        voltage = []
        current = []
        for item in data:
            if item['channelid'] == id:
                timestamps.append(item['timestampsec'])
                rate.append(item['rate'])
                voltage.append(item['voltage'])
                current.append(item['current'])
        
        print "no of points", len(timestamps)
        
        try:
            fig = plt.figure(figsize=(8, 8), dpi=96)
            fig.canvas.set_window_title('PNXT')
            dates = [dt.datetime.fromtimestamp(ts) for ts in np.array(timestamps)]
            ax = plt.gca()
            plt.xticks(rotation = 45)
#             plt.subplots_adjust(bottom = 0.3)
            xfmt = md.DateFormatter('%Y-%m-%d %H:%M')
            ax.xaxis.set_major_formatter(xfmt)
            
            gs = gridspec.GridSpec(3, 1)
            ratePlot = plt.subplot(gs[0, 0])
            fig.add_subplot(ratePlot)
            voltagePlot = plt.subplot(gs[1, 0])
            fig.add_subplot(voltagePlot)
            currentPlot = plt.subplot(gs[2, 0])
            fig.add_subplot(currentPlot)
            
            voltagePlot.set_ylim(40000., 60000.)
            currentPlot.set_ylim(15000., 50000.)       
            
            ratePlot.plot(np.array(dates), np.array(rate), label="rate")
            voltagePlot.plot(np.array(dates), np.array(voltage), color="green", label='voltage')
            currentPlot.plot(np.array(dates), np.array(current), color='red',  label="current")
            
            ratePlot.legend()
            voltagePlot.legend()
            currentPlot.legend()
            
            plt.show()
        
        except Exception as e:
            print e.__doc__
            print e.message
            raise 

no of points 13764


#Plotting Lumi

In [ ]:
fillNo = 0
dateTimeStart = '2016-04-10 13:00'
dateTimeEnd = '2016-04-12 13:30'


tsStart = time.mktime(dt.datetime.strptime(dateTimeStart, "%Y-%m-%d %H:%M").timetuple())
tsEnd = time.mktime(dt.datetime.strptime(dateTimeEnd, "%Y-%m-%d %H:%M").timetuple())

for leaf in f.walk_nodes(where='/',classname="Leaf"):
    if leaf.name == 'radmonlumi':
        data =  f.get_node(where='/',name="radmonlumi", classname="Leaf")
        timestamps = []
        minusz = []
        plusz = []
        for item in data:
            ts = item['timestampsec']
            if ts > tsStart and ts < tsEnd:
                timestamps.append(item['timestampsec'])
                minusz.append(item['minusz'])
                plusz.append(item['plusz'])
                fillno = item['fillnum']
        
        print "no of points", len(timestamps)
        
        try:
            fig = plt.figure(figsize=(8, 6), dpi=96)
            fig.canvas.set_window_title('Radmon Lumi fill ' + str(fillno))
            dates = [dt.datetime.fromtimestamp(ts) for ts in np.array(timestamps)]
            ax = plt.gca()
#             ax.xticks(rotation = 'veritcal')
#             ax.subplots_adjust(bottom = 0.3)
            xfmt = md.DateFormatter('%Y-%m-%d %H:%M')
            ax.xaxis.set_major_formatter(xfmt)
            
            gs = gridspec.GridSpec(2, 1)
            minusPlot = plt.subplot(gs[0, 0])
            fig.add_subplot(minusPlot)
            plusPlot = plt.subplot(gs[1, 0])
            fig.add_subplot(plusPlot)
            
            minusPlot.set_ylim(0., 10.)
            plusPlot.set_ylim(0., 10.)       
            

            minusPlot.plot(np.array(dates), np.array(minusz), label="-Z, Fill " +  str(fillno))
            plusPlot.plot(np.array(dates), np.array(plusz), color="green", label="+Z, Fill " +  str(fillno))
            
#             fig.subplots_adjust(bottom = 0.3)
#             fig.autofmt_xdate()
            
            minusPlot.legend()
            plusPlot.legend()
            
            plt.show()
        
        except Exception as e:
            print e.__doc__
            print e.message
            raise 
            